# Modeling Recommender System with Surprise

In [229]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('fast')

from src.utilities import *
import surprise
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import BaselineOnly, SVD, NMF, KNNBasic, KNNBaseline, KNNWithMeans, NormalPredictor
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy
from collections import defaultdict

In [5]:
ratings_df = csv('https://markg110.s3-us-west-1.amazonaws.com/data/BX-Book-Ratings.csv')
books_df = csv('https://markg110.s3-us-west-1.amazonaws.com/data/BX-Books.csv')
users_df = csv('https://markg110.s3-us-west-1.amazonaws.com/data/BX-Users.csv')

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/

In [6]:
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [7]:
users_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [9]:
print(books_df.shape)
print('')
books_df.head()

(271360, 8)



,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [10]:
books_new = books_df.copy()
books_new.dropna(inplace=True)
columns = ['Book-Author', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']
books_df.drop(columns=columns, inplace=True)
books_df.rename(columns={'Year-Of-Publication':'Publication-Year'},inplace=True)
books_ratings_df = pd.merge(ratings_df, books_df, on='ISBN')
print(books_ratings_df.shape)
print('===========================================================')
books_ratings_df.head()

(1031136, 5)


,User-ID,ISBN,Book-Rating,Book-Title,Publication-Year
0,276725,034545104X,0,Flesh Tones: A Novel,2002
1,2313,034545104X,5,Flesh Tones: A Novel,2002
2,6543,034545104X,0,Flesh Tones: A Novel,2002
3,8680,034545104X,5,Flesh Tones: A Novel,2002
4,10314,034545104X,9,Flesh Tones: A Novel,2002


In [12]:
# Removing irrelevant data and casting publication year to int64
books_ratings_df = books_ratings_df[books_ratings_df['Publication-Year'] != 'Gallimard']
books_ratings_df = books_ratings_df[books_ratings_df['Publication-Year'] != 'DK Publishing Inc']
books_ratings_df['Publication-Year'] = books_ratings_df['Publication-Year'].apply(lambda x: int(x))

In [16]:
# Isolating most rated books which were published between 1975 and 2002
books_ratings_df = books_ratings_df[books_ratings_df['Publication-Year'] >= 1975]
books_ratings_df = books_ratings_df[books_ratings_df['Publication-Year'] <= 2002]
books_ratings_df.shape

(902959, 5)

In [17]:
min_book_ratings = 50 # Books: At least 50 ratings received
min_user_ratings = 50 # Users: At least 50 ratings given

In [19]:
# Trimming down dataset
filter_books = books_ratings_df['ISBN'].value_counts() > min_book_ratings # Isolating books with more than 50 ratings received
filter_books = filter_books[filter_books].index.tolist() # Isolating ISBNs
filter_users = books_ratings_df['User-ID'].value_counts() > min_user_ratings # Isolating users with more than 50 ratings given
filter_users = filter_users[filter_users].index.tolist() # Isolating User-IDs

In [23]:
# Filtering dataframe based on minimum number of ratings by ISBNs and User-IDs
filtered_df = books_ratings_df[(books_ratings_df['ISBN'].isin(filter_books)) & (books_ratings_df['User-ID'].isin(filter_users))]

## Modeling: <code>filtered_df</code>

In [32]:
columns=['Book-Title', 'Publication-Year']
filtered_df.drop(columns=columns, inplace=True)
print(filtered_df.shape)
print('')
filtered_df.head()

(117267, 3)



/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,User-ID,ISBN,Book-Rating
2,6543,034545104X,0
4,10314,034545104X,9
5,23768,034545104X,0
7,28523,034545104X,0
10,56157,034545104X,0


In [33]:
# Instantiate Surprise classes
reader = Reader(rating_scale=(0,10))
data = Dataset.load_from_df(filtered_df, reader)

In [34]:
# Algorithm iterator for cross_validation
algorithm = [BaselineOnly(), SVD(), KNNBasic(), KNNBaseline(), KNNWithMeans(), NormalPredictor()]

In [35]:
comparison_list = []

for algo in algorithm:
    # Perform cross validation with RMSE as evaluation metric
    results = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False)
    
    # Append to comparison list
    get_result = pd.DataFrame.from_dict(results).mean(axis=0)
    get_result = get_result.append(pd.Series([str(algo).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    comparison_list.append(get_result)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

In [233]:
# Generating cross validation results
# We choose BaselineOnly algorithm as it yields the lowest RMSE
surprise_results = pd.DataFrame(comparison_list).set_index('Algorithm').sort_values('test_rmse')
surprise_results.to_csv('surprise_results.csv')
surprise_results

,test_rmse,fit_time,test_time
Algorithm,,,
BaselineOnly,3.329897,0.133036,0.125911
KNNWithMeans,3.419990,0.504006,1.885585
KNNBaseline,3.427019,0.585988,2.249084
SVD,3.512723,3.938870,0.164002
KNNBasic,3.644096,0.466363,1.781149
NormalPredictor,4.665204,0.109652,0.168673


In [37]:
# Configuring baselines using alternating least squares (ALS) 
# and stochastic gradient descent (SGD) for comparison
print('Using SGD: Stochastic Gradient Descent')
bsl_options1 = {'method': 'sgd',
               'learning_rate': 0.00005}

algo1 = BaselineOnly(bsl_options=bsl_options1) # Note algo1: SGD
print('==========================================')

Using SGD: Stochastic Gradient Descent


In [38]:
print('Using ALS: Alternating Least Squares')
bsl_options = {'method':'als', 'n_epochs':5, 'regu_u': 12, 'reg_i':5}

algo = BaselineOnly(bsl_options=bsl_options) # Note algo: ALS

Using ALS: Alternating Least Squares


In [41]:
print('Cross validate SGD')
print('==========================================')
cross_validate(algo1, data, measures=['RMSE'], cv=5, verbose=False)

Cross validate SGD
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...


{'test_rmse': array([3.61970504, 3.62817462, 3.61813693, 3.62072998, 3.6407386 ]),
 'fit_time': (0.2858119010925293,
  0.28494977951049805,
  0.28933119773864746,
  0.2842090129852295,
  0.2846250534057617),
 'test_time': (0.08953022956848145,
  0.2155609130859375,
  0.08902454376220703,
  0.2149209976196289,
  0.09046268463134766)}

In [42]:
# Alternating Least Squares returns lower array of test RMSE
print('Cross validate ALS')
print('==========================================')
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False)

Cross validate ALS
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([3.31228886, 3.3554559 , 3.33619559, 3.35197517, 3.32144036]),
 'fit_time': (0.1008291244506836,
  0.10027408599853516,
  0.10046601295471191,
  0.09932065010070801,
  0.10054135322570801),
 'test_time': (0.21363592147827148,
  0.08981657028198242,
  0.21383142471313477,
  0.08935928344726562,
  0.09006977081298828)}

In [46]:
# Train, test, split to generate ratings predictions of each book
trainset, testset = train_test_split(data, test_size=0.20)
algo = BaselineOnly(bsl_options=bsl_options)
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 3.3504


3.3503985653366954

## Predictions

In [52]:
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0

In [53]:
df_pred = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df_pred['Iu']=df_pred['uid'].apply(get_Iu)
df_pred['Ui']=df_pred['iid'].apply(get_Ui)
df_pred['err']=abs(df_pred.est - df_pred.rui)

print('Shape of df_pred:', df_pred.shape)
print('==========================================')
print('')
print('Predictions Dataframe:')
print('==========================================')
print('')
df_pred.head()

Shape of df_pred: (23454, 8)

Predictions Dataframe:



,uid,iid,rui,est,details,Iu,Ui,err
0,212898,0425161242,0.0,0.000000,{'was_impossible': False},195,44,0.000000
1,238120,0140119906,0.0,0.217721,{'was_impossible': False},293,78,0.217721
2,11676,0440225299,10.0,3.731917,{'was_impossible': False},1059,39,6.268083
3,162590,0440226104,0.0,1.610351,{'was_impossible': False},33,124,1.610351
4,105979,0385492081,8.0,1.788216,{'was_impossible': False},155,134,6.211784


In [69]:
# Top 20 Best Predictions
print('''
uid - user id
iid - isbn
rui - actual user rating
est - predicted ratings
Iu -  number of items rated by given user
Ui - number of users that have rated given item
''')
best_predictions = df_pred.sort_values(by='err', ascending=True)[:20]
best_predictions.sort_values(by='Ui', ascending=False)


uid - user id
iid - isbn
rui - actual user rating
est - predicted ratings
Iu -  number of items rated by given user
Ui - number of users that have rated given item



,uid,iid,rui,est,details,Iu,Ui,err
14313,35050,0446605484,0.0,0.0,{'was_impossible': False},143,103,0.0
1688,98741,014023313X,0.0,0.0,{'was_impossible': False},219,85,0.0
1655,175003,0671004530,0.0,0.0,{'was_impossible': False},264,72,0.0
8099,76352,042513699X,0.0,0.0,{'was_impossible': False},444,69,0.0
8127,98741,0786889020,0.0,0.0,{'was_impossible': False},219,58,0.0
21508,127429,0140254544,0.0,0.0,{'was_impossible': False},120,56,0.0
13855,55492,0446605581,0.0,0.0,{'was_impossible': False},332,50,0.0
8026,269566,0345404769,0.0,0.0,{'was_impossible': False},272,49,0.0
0,212898,0425161242,0.0,0.0,{'was_impossible': False},195,44,0.0
1794,55492,0515113328,0.0,0.0,{'was_impossible': False},332,41,0.0


In [76]:
# Top 20 Worst Predictions
print('''
uid - user id
iid - isbn
rui - actual user rating
est - predicted ratings
Iu -  number of items rated by given user
Ui - number of users that have rated given item
''')
worst_predictions = df_pred.sort_values(by='err', ascending=False).head(20)
worst_predictions
# worst_predictions.sort_values(by='Ui', ascending=False)


uid - user id
iid - isbn
rui - actual user rating
est - predicted ratings
Iu -  number of items rated by given user
Ui - number of users that have rated given item



,uid,iid,rui,est,details,Iu,Ui,err
3960,31826,0439136350,0.0,10.000000,{'was_impossible': False},75,73,10.000000
5236,26544,055358264X,10.0,0.000000,{'was_impossible': False},206,48,10.000000
12331,14897,0515099546,10.0,0.000000,{'was_impossible': False},26,30,10.000000
13502,217375,0553569031,10.0,0.054394,{'was_impossible': False},211,52,9.945606
14028,11601,0375703063,10.0,0.133850,{'was_impossible': False},187,62,9.866150
13037,55548,0553278398,10.0,0.267914,{'was_impossible': False},119,23,9.732086
11313,77940,0671027581,10.0,0.342740,{'was_impossible': False},43,45,9.657260
12587,217375,0446364193,10.0,0.348167,{'was_impossible': False},211,135,9.651833
14941,205735,0373825013,10.0,0.370728,{'was_impossible': False},75,69,9.629272
21970,238781,0345443284,10.0,0.382054,{'was_impossible': False},169,149,9.617946


# Top Recommendations for Users with Book Title

In [77]:
def get_top3_recommendations(predictions, topN = 3):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]
     
    return top_recs

In [83]:
top_preds=get_top3_recommendations(predictions, topN=3)

In [99]:
book_list = []
for uid, user_ratings in top_preds.items():
    book_list.append([uid, [iid for (iid, _) in user_ratings]])

In [102]:
book_list_df=pd.DataFrame(book_list)

In [104]:
book_list_df.rename(columns={0:'uid', 1:'iid'}, inplace=True)

In [183]:
book_list_df.rename(columns={'uid':'User-ID', 'iid':'ISBN'},inplace=True)
book_list_df.head()

,User-ID,ISBN
0,212898,"[0064471101, 0345435249, 0451176464]"
1,238120,"[0439136350, 0679457526, 0060920084]"
2,11676,"[0439064864, 0618002227, 0894803204]"
3,162590,"[0451153553, 0345351525, 0440226104]"
4,105979,"[0805063889, 0140053204, 0385498721]"


In [189]:
test_df = pd.DataFrame(book_list_df['ISBN'].values.tolist()[0])
test_df.head()

,0
0,0064471101
1,0345435249
2,0451176464


In [205]:
first_five = book_list_df.head()

In [206]:
first_five[['Book1', 'Book2','Book3']] = pd.DataFrame(first_five.ISBN.values.tolist(), index=first_five.index)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [212]:
first_five

,User-ID,ISBN,Book1,Book2,Book3
0,212898,"[0064471101, 0345435249, 0451176464]",0064471101,0345435249,0451176464
1,238120,"[0439136350, 0679457526, 0060920084]",0439136350,0679457526,0060920084
2,11676,"[0439064864, 0618002227, 0894803204]",0439064864,0618002227,0894803204
3,162590,"[0451153553, 0345351525, 0440226104]",0451153553,0345351525,0440226104
4,105979,"[0805063889, 0140053204, 0385498721]",0805063889,0140053204,0385498721


In [213]:
first_five.drop('ISBN',1,inplace=True)
first_five

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,User-ID,Book1,Book2,Book3
0,212898,0064471101,0345435249,0451176464
1,238120,0439136350,0679457526,0060920084
2,11676,0439064864,0618002227,0894803204
3,162590,0451153553,0345351525,0440226104
4,105979,0805063889,0140053204,0385498721


In [214]:
books_ratings_df.head()

,User-ID,ISBN,Book-Rating,Book-Title,Publication-Year
0,276725,034545104X,0,Flesh Tones: A Novel,2002
1,2313,034545104X,5,Flesh Tones: A Novel,2002
2,6543,034545104X,0,Flesh Tones: A Novel,2002
3,8680,034545104X,5,Flesh Tones: A Novel,2002
4,10314,034545104X,9,Flesh Tones: A Novel,2002


## A for-loop would prove to be more efficient

In [ ]:
# books = ['Book1','Book2','Book3','Book4','Book5']
# for i in books:
#     first_five=pd.merge(first_five, books_ratings_df[['ISBN', 'Book-Title']],
#                        left_on=i, right_on='ISBN', how='left')
# rec_df = first_five.drop_duplicates()
# rec_df[['User-ID', 'Book-Title_x', 'Book-Title_y', 'Book-Title']]

In [215]:
first_five = pd.merge(first_five, books_ratings_df[['ISBN', 'Book-Title']],
                     left_on='Book1', right_on='ISBN', how='left')

In [217]:
first_five = pd.merge(first_five, books_ratings_df[['ISBN', 'Book-Title']],
                     left_on='Book2', right_on='ISBN', how='left')

In [219]:
first_five = pd.merge(first_five, books_ratings_df[['ISBN', 'Book-Title']],
                     left_on='Book3', right_on='ISBN', how='left')

# TOP 3 RECOMMENDED BOOKS FOR FIRST 5 RANDOM USERS

In [235]:
rec_df = first_five.drop_duplicates()
rec_df[['User-ID', 'Book-Title_x', 'Book-Title_y', 'Book-Title']].to_csv('top3.csv')
rec_df[['User-ID', 'Book-Title_x', 'Book-Title_y', 'Book-Title']]

,User-ID,Book-Title_x,Book-Title_y,Book-Title
0,212898,The Magician's Nephew (rack) (Narnia),Darwin's Radio,Gerald's Game
600990,238120,Harry Potter and the Prisoner of Azkaban (Book 3),Into Thin Air : A Personal Account of the Moun...,The Lost Continent: Travels in Small-Town America
1590915,11676,Harry Potter and the Chamber of Secrets (Book 2),The Fellowship of the Ring (The Lord of the Ri...,The Book of Questions
2319195,162590,Misery,The Queen of the Damned (Vampire Chronicles (P...,Black and Blue
9929889,105979,Nickel and Dimed: On (Not) Getting By in America,Travels With Charley: In Search of America,Survivor : A Novel
